In [2]:
from bs4 import BeautifulSoup
import urllib
import urllib2
import pandas as pd
import numpy as np
import re
from string import digits
import requests

In [3]:
# read in url

tmp = requests.get('https://fitness-websites.no1reviews.com/')
c = tmp.content
soup = BeautifulSoup(c)

/Users/tori.potruch/miniconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/tori.potruch/miniconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
i = 1
ranks = pd.DataFrame()
for s in soup.find_all(class_='ProductName'):
    #print s
    rank = i
    name = s.get_text()
    #print str(rank) + '. ' + name
    pos = str(s).find('href')
    pos2 = str(s).find('style')
    url = str(s)[pos+6:pos2-2]
    #print url
    review_url = 'https://fitness-websites.no1reviews.com/' + url
    feature_url = 'https://fitness-websites.no1reviews.com/features/' + url
    i = i+1
    temp = pd.DataFrame({"Rank": [rank], "Site Name": [name], "Url":[url], "Review Url":[review_url], \
                        'Feature Url':[feature_url]})
    temp = temp[['Rank','Site Name','Url','Review Url','Feature Url']]
    ranks = ranks.append(temp,ignore_index=True)
ranks

,Rank,Site Name,Url,Review Url,Feature Url
0,1,Jillian Michaels,jillianmichaels.html,https://fitness-websites.no1reviews.com/jillia...,https://fitness-websites.no1reviews.com/featur...
1,2,Daily Burn,dailyburn.html,https://fitness-websites.no1reviews.com/dailyb...,https://fitness-websites.no1reviews.com/featur...
2,3,Fitocracy,fitocracy.html,https://fitness-websites.no1reviews.com/fitocr...,https://fitness-websites.no1reviews.com/featur...
3,4,Fitness Blender,fitnessblender.html,https://fitness-websites.no1reviews.com/fitnes...,https://fitness-websites.no1reviews.com/featur...
4,5,Nerd Fitness,nerdfitness.html,https://fitness-websites.no1reviews.com/nerdfi...,https://fitness-websites.no1reviews.com/featur...
5,6,Train Online,train-online.html,https://fitness-websites.no1reviews.com/train-...,https://fitness-websites.no1reviews.com/featur...
6,7,Beachbody,beachbody.html,https://fitness-websites.no1reviews.com/beachb...,https://fitness-websites.no1reviews.com/featur...
7,8,Muscle And Fitness,muscleandfitness.html,https://fitness-websites.no1reviews.com/muscle...,https://fitness-websites.no1reviews.com/featur...
8,9,Fitness.com,fitness.html,https://fitness-websites.no1reviews.com/fitnes...,https://fitness-websites.no1reviews.com/featur...
9,10,Changing Shape,changing-shape.html,https://fitness-websites.no1reviews.com/changi...,https://fitness-websites.no1reviews.com/featur...


In [5]:
ratings = pd.DataFrame()
for s in soup.find_all(class_='editors_big_rating_bar'):
    pos = str(s).find('width')
    pos2 = str(s).find('><')
    rating = str(s)[pos+6:pos2-2]
    temp = pd.DataFrame({"Overall Rating":[rating]})
    ratings = ratings.append(temp,ignore_index=True)
ranks = pd.concat([ranks,ratings],axis=1)
    

In [6]:
i=0
url = ranks['Review Url'][i]
tmp2 = requests.get(url)
c2 = tmp2.content
soup2 = BeautifulSoup(c2)

editor_sections = pd.DataFrame()
editor_ratings = pd.DataFrame()

for s in soup2.find_all(style="font-weight:normal"):
    tmp = s.get_text()[:-1]
    sections = pd.DataFrame({"Rating Section":[tmp]})
    editor_sections = editor_sections.append(sections,ignore_index=True)

for s in soup2.find_all(class_='editors_small_rating_bar'):
    tmp = str(s)
    pos = tmp.find('width')
    pos2 = tmp.find('><')
    rating = tmp[pos+6:pos2-2]   
    ratings = pd.DataFrame({'Rating':[rating]})
    editor_ratings = editor_ratings.append(ratings,ignore_index=True)

editor_ratings = editor_ratings.iloc[1:-1].reset_index(drop=True)

editor_final = pd.concat([editor_sections,editor_ratings],axis=1)
editor_final = editor_final.set_index('Rating Section').T

for i in range(1,len(ranks)):
    url = ranks['Review Url'][i]
    tmp2 = requests.get(url)
    c2 = tmp2.content
    soup2 = BeautifulSoup(c2)

    editor_sections = pd.DataFrame()
    editor_ratings = pd.DataFrame()

    for s in soup2.find_all(style="font-weight:normal"):
        tmp = s.get_text()[:-1]
        sections = pd.DataFrame({"Rating Section":[tmp]})
        editor_sections = editor_sections.append(sections,ignore_index=True)

    for s in soup2.find_all(class_='editors_small_rating_bar'):
        tmp = str(s)
        pos = tmp.find('width')
        pos2 = tmp.find('><')
        rating = tmp[pos+6:pos2-2]   
        ratings = pd.DataFrame({'Rating':[rating]})
        editor_ratings = editor_ratings.append(ratings,ignore_index=True)

    editor_ratings = editor_ratings.iloc[1:-1].reset_index(drop=True)
    
    final_tmp = pd.concat([editor_sections,editor_ratings],axis=1)
    final_tmp = final_tmp.set_index('Rating Section').T
    editor_final = editor_final.append(final_tmp,ignore_index=True)

In [7]:
final = pd.concat([ranks,editor_final],axis=1)
final

,Rank,Site Name,Url,Review Url,Feature Url,Overall Rating,Exercise Demos and Instructions,Workouts Customized by User,Workouts Customized by Trainer,Regular Personal Support,Tools for Tracking Progress,Website Design and Ease of Use,Value for Money
0,1,Jillian Michaels,jillianmichaels.html,https://fitness-websites.no1reviews.com/jillia...,https://fitness-websites.no1reviews.com/featur...,100,100,90,90,100,100,100,100
1,2,Daily Burn,dailyburn.html,https://fitness-websites.no1reviews.com/dailyb...,https://fitness-websites.no1reviews.com/featur...,90,80,80,90,90,100,90,100
2,3,Fitocracy,fitocracy.html,https://fitness-websites.no1reviews.com/fitocr...,https://fitness-websites.no1reviews.com/featur...,80,80,70,100,100,80,80,80
3,4,Fitness Blender,fitnessblender.html,https://fitness-websites.no1reviews.com/fitnes...,https://fitness-websites.no1reviews.com/featur...,80,100,100,80,50,70,80,80
4,5,Nerd Fitness,nerdfitness.html,https://fitness-websites.no1reviews.com/nerdfi...,https://fitness-websites.no1reviews.com/featur...,80,70,60,80,90,90,80,80
5,6,Train Online,train-online.html,https://fitness-websites.no1reviews.com/train-...,https://fitness-websites.no1reviews.com/featur...,80,90,70,70,60,80,80,90
6,7,Beachbody,beachbody.html,https://fitness-websites.no1reviews.com/beachb...,https://fitness-websites.no1reviews.com/featur...,80,100,60,100,70,60,80,60
7,8,Muscle And Fitness,muscleandfitness.html,https://fitness-websites.no1reviews.com/muscle...,https://fitness-websites.no1reviews.com/featur...,70,100,60,80,40,50,80,100
8,9,Fitness.com,fitness.html,https://fitness-websites.no1reviews.com/fitnes...,https://fitness-websites.no1reviews.com/featur...,70,70,90,70,60,60,80,80
9,10,Changing Shape,changing-shape.html,https://fitness-websites.no1reviews.com/changi...,https://fitness-websites.no1reviews.com/featur...,70,40,70,60,60,90,100,NaN
